In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shubhambaid/skin-burn-dataset")

print("Path to dataset files:", path)

100%|██████████| 16.6M/16.6M [00:00<00:00, 144MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shubhambaid/skin-burn-dataset/versions/1


In [31]:
import zipfile

print("Downloading dataset from KaggleHub...")
dataset_path = kagglehub.dataset_download("shubhambaid/skin-burn-dataset")
print("Dataset downloaded at:", dataset_path)

if dataset_path.endswith(".zip"):
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        extract_path = dataset_path.rstrip('.zip')
        zip_ref.extractall(extract_path)
    dataset_path = extract_path

print("Folders inside dataset:", os.listdir(dataset_path))

image_dir = os.path.join(dataset_path, 'images')  # Update if folder name different
label_dir = os.path.join(dataset_path, 'labels')  # Update if folder name different

print(f"Image directory: {image_dir}")
print(f"Label directory: {label_dir}")


Dataset downloaded at: /kaggle/input/skin-burn-dataset
Folders inside dataset: ['img560.txt', 'img1194.jpg', 'img275.txt', 'img966.txt', 'img516.jpg', 'img943.txt', 'img254.jpg', 'img263.jpg', 'img699.txt', 'img659.jpg', 'img338.txt', 'img1387.txt', 'img575.jpg', 'img483.txt', 'img4.txt', 'img470.txt', 'img794.txt', 'img1013.jpg', 'img0.jpg', 'img988.txt', 'img66.txt', 'img1119.jpg', 'img403.jpg', 'img1138.jpg', 'img1280.txt', 'img1173.jpg', 'img786.txt', 'img522.txt', 'img1176.jpg', 'img696.txt', 'img1343.txt', 'img932.txt', 'img822.txt', 'img1120.txt', 'img224.jpg', 'img985.jpg', 'img113.txt', 'img514.jpg', 'img170.jpg', 'img212.jpg', 'img18.txt', 'img280.jpg', 'img344.txt', 'img341.txt', 'img1303.txt', 'img373.txt', 'img77.jpg', 'img340.jpg', 'img521.jpg', 'img154.jpg', 'img170.txt', 'img498.jpg', 'img226.txt', 'img42.jpg', 'img989.txt', 'img317.txt', 'img1144.txt', 'img921.txt', 'img1285.jpg', 'img1210.txt', 'img920.txt', 'img435.txt', 'img342.jpg', 'img691.jpg', 'img286.jpg', 'img

In [32]:
import os
import numpy as np
from PIL import Image

def bbox_to_mask(bboxes, img_width, img_height, target_size=(128,128)):
    """
    Convert list of bounding boxes to mask.
    bboxes: list of [x_min, y_min, x_max, y_max] in original image scale
    """
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    for bbox in bboxes:
        x_min, y_min, x_max, y_max = bbox
        x_min = max(0, int(x_min))
        y_min = max(0, int(y_min))
        x_max = min(img_width, int(x_max))
        y_max = min(img_height, int(y_max))
        mask[y_min:y_max, x_min:x_max] = 1
    # Resize mask to target size
    mask = Image.fromarray(mask)
    mask = mask.resize(target_size, resample=Image.NEAREST)
    return np.array(mask)

def load_data(dataset_path, target_size=(128,128)):
    files = os.listdir(dataset_path)
    image_files = [f for f in files if f.endswith('.jpg')]
    images = []
    masks = []

    for img_file in image_files:
        # Load image
        img_path = os.path.join(dataset_path, img_file)
        img = Image.open(img_path).convert('RGB')
        orig_w, orig_h = img.size
        img = img.resize(target_size)
        img = np.array(img) / 255.0  # Normalize to [0,1]

        # Load corresponding annotation file (bbox)
        label_file = img_file.replace('.jpg', '.txt')
        label_path = os.path.join(dataset_path, label_file)
        bboxes = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    # Assuming label format is: class_id x_center y_center width height (normalized)
                    parts = line.strip().split()
                    if len(parts) == 5:
                        _, x_c, y_c, w, h = map(float, parts)
                        # Convert normalized coords to pixel bbox
                        x_min = (x_c - w/2) * orig_w
                        y_min = (y_c - h/2) * orig_h
                        x_max = (x_c + w/2) * orig_w
                        y_max = (y_c + h/2) * orig_h
                        bboxes.append([x_min, y_min, x_max, y_max])

        # Convert bbox list to mask of target_size
        mask = bbox_to_mask(bboxes, orig_w, orig_h, target_size=target_size)
        mask = np.expand_dims(mask, axis=-1)  # Add channel dim

        images.append(img)
        masks.append(mask)

    return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)


In [33]:
dataset_path = "/kaggle/input/skin-burn-dataset"
IMG_SIZE = (128,128)

X, Y = load_data(dataset_path, target_size=IMG_SIZE)
print("Images shape:", X.shape)
print("Masks shape:", Y.shape)


Images shape: (1225, 128, 128, 3)
Masks shape: (1225, 128, 128, 1)


In [34]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
import kagglehub

# Step 1: Download dataset from KaggleHub
print("Downloading dataset from KaggleHub...")
dataset_path = kagglehub.dataset_download("shubhambaid/skin-burn-dataset")
print(f"Dataset downloaded at: {dataset_path}")

# Step 2: Helper functions

def bbox_to_mask(bboxes, img_width, img_height, target_size=(128,128)):
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    for bbox in bboxes:
        x_min, y_min, x_max, y_max = bbox
        x_min = max(0, int(x_min))
        y_min = max(0, int(y_min))
        x_max = min(img_width, int(x_max))
        y_max = min(img_height, int(y_max))
        mask[y_min:y_max, x_min:x_max] = 1
    # Resize mask to target size
    mask = Image.fromarray(mask)
    mask = mask.resize(target_size, resample=Image.NEAREST)
    return np.array(mask)

def load_data(dataset_path, target_size=(128,128)):
    files = os.listdir(dataset_path)
    image_files = [f for f in files if f.endswith('.jpg')]
    images = []
    masks = []

    for img_file in image_files:
        # Load image
        img_path = os.path.join(dataset_path, img_file)
        img = Image.open(img_path).convert('RGB')
        orig_w, orig_h = img.size
        img = img.resize(target_size)
        img = np.array(img) / 255.0  # Normalize to [0,1]

        # Load corresponding annotation file (bbox)
        label_file = img_file.replace('.jpg', '.txt')
        label_path = os.path.join(dataset_path, label_file)
        bboxes = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) == 5:
                        _, x_c, y_c, w, h = map(float, parts)
                        x_min = (x_c - w/2) * orig_w
                        y_min = (y_c - h/2) * orig_h
                        x_max = (x_c + w/2) * orig_w
                        y_max = (y_c + h/2) * orig_h
                        bboxes.append([x_min, y_min, x_max, y_max])

        mask = bbox_to_mask(bboxes, orig_w, orig_h, target_size=target_size)
        mask = np.expand_dims(mask, axis=-1)  # Add channel dim

        images.append(img)
        masks.append(mask)

    return np.array(images, dtype=np.float32), np.array(masks, dtype=np.float32)

# Step 3: Load dataset
IMG_SIZE = (128,128)
X, Y = load_data(dataset_path, target_size=IMG_SIZE)
print(f"Loaded {len(X)} images and masks.")
print(f"Images shape: {X.shape}, Masks shape: {Y.shape}")

# Step 4: Split dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)
print(f"Train size: {len(X_train)}, Val size: {len(X_val)}, Test size: {len(X_test)}")

# Step 5: Prepare tf.data.Dataset
BATCH_SIZE = 16

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=100).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Step 6: Define DeeplabV3+ model
def DeeplabV3Plus(input_shape=(128,128,3), num_classes=1):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]
    layers_output = [base_model.get_layer(name).output for name in layer_names]

    encoder_output = layers_output[-1]  # 4x4

    def aspp_block(x, filters=256, rate=1):
        x = layers.Conv2D(filters, 3, padding='same', dilation_rate=rate, use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        return x

    aspp1 = aspp_block(encoder_output, 256, rate=1)
    aspp6 = aspp_block(encoder_output, 256, rate=6)
    aspp12 = aspp_block(encoder_output, 256, rate=12)
    aspp18 = aspp_block(encoder_output, 256, rate=18)

    aspp = layers.Concatenate()([aspp1, aspp6, aspp12, aspp18])
    aspp = layers.Conv2D(256, 1, padding='same', use_bias=False)(aspp)
    aspp = layers.BatchNormalization()(aspp)
    aspp = layers.ReLU()(aspp)
    aspp = layers.Dropout(0.5)(aspp)

    x = layers.UpSampling2D(size=(4,4), interpolation='bilinear')(aspp)  # 4->16

    low_level_feat = layers_output[1]  # block_3_expand_relu
    low_level_feat = layers.Conv2D(48, 1, padding='same', use_bias=False)(low_level_feat)
    low_level_feat = layers.BatchNormalization()(low_level_feat)
    low_level_feat = layers.ReLU()(low_level_feat)
    low_level_feat = layers.MaxPooling2D(pool_size=(2,2))(low_level_feat)  # 32->16

    x = layers.Concatenate()([x, low_level_feat])

    x = layers.Conv2D(256, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(256, 3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.UpSampling2D(size=(8,8), interpolation='bilinear')(x)  # 16->128

    outputs = layers.Conv2D(num_classes, 1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

# Step 7: Compile and train the model
model = DeeplabV3Plus(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

EPOCHS = 10
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Step 8: Evaluate on test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Dataset downloaded at: /kaggle/input/skin-burn-dataset
Loaded 1225 images and masks.
Images shape: (1225, 128, 128, 3), Masks shape: (1225, 128, 128, 1)
Train size: 857, Val size: 184, Test size: 184


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer_5[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 6,358,017 (24.25 MB)

 Trainable params: 6,323,425 (24.12 MB)

 Non-trainable params: 34,592 (135.12 KB)

Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 678s 11s/step - accuracy: 0.7474 - loss: 0.5713 - val_accuracy: 0.8049 - val_loss: 0.5827
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 566s 10s/step - accuracy: 0.8438 - loss: 0.3373 - val_accuracy: 0.7325 - val_loss: 0.5954
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 575s 11s/step - accuracy: 0.8638 - loss: 0.3038 - val_accuracy: 0.4710 - val_loss: 1.9787
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 616s 11s/step - accuracy: 0.8652 - loss: 0.2995 - val_accuracy: 0.8204 - val_loss: 0.3749
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 617s 10s/step - accuracy: 0.8712 - loss: 0.2806 - val_accuracy: 0.5720 - val_loss: 1.0493
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 563s 10s/step - accuracy: 0.8845 - loss: 0.2585 - val_accuracy: 0.5631 - val_loss: 1.2665
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 622s 10s/step - accuracy: 0.8875 - loss: 0.2508 - val_accuracy: 0.6146 - val_loss: 0.7808
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 621s 10s/step - accuracy: 0.8967 - loss: 0.2350 - val_accuracy: 0.